In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# NCJ notebook, VGIS9 research mini project.
# Original baseline kernel: https://www.kaggle.com/stpeteishii/landmark-recognition-conv2d

# Imports for Deep Learning
# baseline cnn model for mnist
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
# import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Dense,Flatten,Dropout,Conv2D,MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import cv2

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#   for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

print('Starting up NC notebook')

In [ ]:
# Loading dataset into initial variable
train_dir = '../input/landmark-recognition-2021/train'
test_dir = '../input/landmark-recognition-2021/test'
train=pd.read_csv('../input/landmark-recognition-2021/train.csv')
train

1580470 images exsist in the dataset

In [ ]:
# Getting landmark numberr
Name=[]
for landmark in train['landmark_id'].unique():
    Name+=[landmark]
print(len(Name))
n=len(Name)

# Mapping directory
N=list(range(n))
normal_mapping=dict(zip(Name,N)) 
reverse_mapping=dict(zip(N,Name)) 

81313 different classes exsist in the dataset, lanmark_id skips some numbers from 0 to 203092

In [ ]:
trainX0=[]
trainY0=[]
# Loop for making train image array and train labels.
for i in tqdm(range(25000)):   #Change to decide number of images
    idt=train.loc[i,'id']+'.jpg'
    landt=train.loc[i,'landmark_id']
    path=os.path.join(train_dir,idt[0],idt[1],idt[2])
    image=cv2.imread(os.path.join(path,idt))
    image2=cv2.resize(image,dsize=(128,128),interpolation=cv2.INTER_CUBIC)  #Size is set to 128*128
    trainX0+=[image2]
    trainY0+=[landt] 

In [ ]:
# Finding classes with less than 5, and between 5 and 10 images
less_5_val = 0
between_5_and_10_val = 0
for i in tqdm(range(203092)):
    val = len(train.loc[train['landmark_id'] == i])
    if(val < 5 and val > 0):
        less_5_val = less_5_val+1
    elif(val >= 5 and val <= 10):
        between_5_and_10_val = between_5_and_10_val+1

In [ ]:
print(less_5_val)
print(between_5_and_10_val)

17297 classes have less than five images in them, and 27348 classes has between 5 and 10 images

In [ ]:
# Histogram of instances per class

hist = plt.figure(figsize = (10, 10))
ax = plt.hist(train["landmark_id"], bins = train["landmark_id"].unique())
plt.ylim([0, 100])
plt.show()

In [ ]:
# Plotting 4 * 4 images from different classes
def plot_four_im(im1,im2,im3,im4): 

    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
    fig
    ax1.imshow(trainX0[im1])
    fig
    ax2.imshow(trainX0[im2])
    fig
    ax3.imshow(trainX0[im3])
    fig
    ax4.imshow(trainX0[im4])
    
    
def plot_im(indices,size_x,size_y): 

    fig = plt.figure(figsize=(10, 7))
    
    
print(train.loc[800:803,'landmark_id'])
plot_four_im(800,801,802,803)

print(train.loc[0:3,'landmark_id'])
plot_four_im(0,1,2,3)

print(train.loc[467:470,'landmark_id'])
plot_four_im(467,468,469,470)

print(train.loc[55:58,'landmark_id'])
plot_four_im(55,56,57,58)

Now starts training of model, and testing of the model

In [ ]:
#Image augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from numpy import expand_dims

trainX0_augmented=[]

#Random Rotation
#datagen = ImageDataGenerator(rotation_range=180)

#Random Zoom augmentation
#datagen = ImageDataGenerator(zoom_range=[0.3,1.0])

#Random brightness augmentation
datagen = ImageDataGenerator(brightness_range=[0.3,1.0])


for x in tqdm(range(25000)):
    
    img = trainX0[x]
    sample = expand_dims(img, 0)
    it = datagen.flow(sample, batch_size=1)
    batch = it.next()
    image = batch[0].astype('uint8')
    trainX0_augmented+=[image]

    #print(trainX0_rotate)

plt.imshow(trainX0_augmented[1])
trainX0 = trainX0_augmented


In [ ]:
# Prepare training data and test data, and the corresponding labels
trainX=np.array(trainX0)

trainX=np.array(trainX0)
trainY1=pd.Series(trainY0).map(normal_mapping)
trainY2=np.array(trainY1)

trainY3=to_categorical(trainY2)
X_train=np.array(trainX0).reshape(-1,128,128,3)
y_train=np.array(trainY3)

trainx,testx,trainy,testy=train_test_split(X_train,y_train,test_size=0.3,random_state=44) # Data is split into 70% training images mand 30% test images

print(trainx.shape)
print(testx.shape)
print(trainy.shape)
print(testy.shape)
print(trainy.shape[1])

In [ ]:
trainY3.shape

In [ ]:
# Building model
model = Sequential()
model.add(Conv2D(32,(2,2),input_shape = (128,128,3),activation = 'relu'))
model.add(Conv2D(32, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(trainy.shape[1], activation='softmax'))

#model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy']) 
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-4),metrics=['accuracy']) 

model.summary()

In [ ]:
his = model.fit(trainx, trainy, validation_split=0.2, epochs=20, batch_size=128, verbose=1) 


In [ ]:
get_acc = his.history['accuracy']
value_acc = his.history['val_accuracy']
get_loss = his.history['loss']
validation_loss = his.history['val_loss']

epochs = range(len(get_acc))
plt.plot(epochs, get_acc, 'r', label='Accuracy of Training data')
plt.plot(epochs, value_acc, 'b', label='Accuracy of Validation data')
plt.title('Training vs validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
epochs = range(len(get_loss))
plt.plot(epochs, get_loss, 'r', label='Loss of Training data')
plt.plot(epochs, validation_loss, 'b', label='Loss of Validation data')
plt.title('Training vs validation loss')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
# Exploring classification results

y_pred=model.predict(testx)  # Predict classes of each test image
pred=np.argmax(y_pred,axis=1)  # Make prediction list
ground = np.argmax(testy,axis=1) # Make ground truth label
#print(classification_report(ground,pred))


#imgplot = plt.imshow(testx[1])

# Find images locations with the supplied label
Indices_ground = np.where(ground==9)
print(Indices_ground)

# Find images locations of a class, predicted by model
Indices_pred = np.where(pred==9)
print(Indices_pred)

# Print number of images per class for comparison
print(len(Indices_pred[0]))
print(len(Indices_ground[0]))

#print(pred[0:20])
imgplot = plt.imshow(testx[3494])
#plot_figs(Indices_ground)


# Show images to put into report
print(ground[9])

plt.imshow(testx[7211])

"""
# Show images to put into report
fig, axis = plt.subplots(2,3)
fig
axis[0,0].imshow(testx[424])
fig
axis[0,1].imshow(testx[1399])
fig
axis[0,2].imshow(testx[2242])
fig
axis[1,0].imshow(testx[2602])
fig
axis[1,1].imshow(testx[6408])
fig
axis[1,2].imshow(testx[6711])


#print(pred[9]);


#

#print(len(Indices_pred[0]))
#imgplot = plt.imshow(testx[56])
#print(ground[99])
#imgplot = plt.imshow(testx[706])

#imgplot = plt.imshow(testx[1463])

print(pred[1640]); print(ground[1783])

"""


In [ ]:
# Print prediction report of each class. Overall accuracy in the bottom. Explore the results to see what sticks out
print(classification_report(ground,pred))
